In [1]:
!python --version

Python 3.8.15


In [2]:
!python -c "import torch; print(torch.__version__)"

1.12.1+cu113


In [3]:
!python -c "import torch; print(torch.version.cuda)"

11.3


In [4]:
!pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.12.1+cu113.html
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 1.9 MB 31.2 MB/s 
     |████████████████████████████████| 8.9 MB 69.0 MB/s 
     |████████████████████████████████| 3.5 MB 73.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 564 kB 25.2 MB/s 
     |████████████████████████████████| 280 kB 76.4 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773302 sha256=1a1c75f843c9799a2371d6c81978bf8226af75a52b2946d1008f6f515aa3ec3b
  Stored in directory: /root/.cache/pip/wheels/59/a3/20/198928106d3169865ae73afcbd3d3d1796cf6b429b55c65378
Successfully built torch-geometric
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninsta

In [89]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GraphNet(nn.Module):
    def __init__(self):
      super(GraphNet, self).__init__()
      self.conv1 = GCNConv(64, 32) # first skip through conv1
      self.shortcut1 = nn.Conv1d(64, 32, kernel_size=1, stride=1, bias=False)
      
      self.conv2 = GCNConv(32, 16)
      
      
      self.conv3 = GCNConv(16, 16) # second skip through conv3
      self.shortcut2 = nn.Conv1d(16, 16, kernel_size=1, stride=1, bias=False)
      
      self.conv4 = GCNConv(16, 16)
      
      # self.fc1 = nn.Linear(1026*64, 4096)
      self.fc2 = nn.Linear(1026*16, 1026)

    # x represents our data
    def forward(self, x, edge_index):
      # Flatten x with start_dim=1
      dim0, dim1, dim2 = x.shape[0], x.shape[1], x.shape[2] #[16, 1026, 64]
      input = x.reshape(dim0*dim1, -1, 1)
      x = x.reshape(dim0*dim1,-1) #[16*1026, 64]
      # print('input shape = ', input.shape) #(1026, 16*64)
      # print('edge_index shape = ', edge_index.shape)
      x = F.relu(self.conv1(x, edge_index)) #[16*1026, 32]
      x = F.dropout(x, training=self.training)
      # print('x shape = ', x.shape) #(1026, 512)
      skip = self.shortcut1(input).reshape(dim0*dim1,-1) #[dim0*dim1, 32]
      x2 = self.conv2(x+skip, edge_index) 
      x2 = F.relu(x2) #[16*1026, 16]
      # print('x2 = ', x2.shape)
      x3 = F.relu(self.conv3(x2, edge_index))
      skip2 = self.shortcut2(x2.reshape(dim0*dim1, -1, 1)).reshape(dim0*dim1,-1)

      x4 = F.relu(self.conv4(x3+skip2, edge_index).reshape(dim0,-1))
      # print(x.shape)
      x = torch.flatten(x4, 1) #[16,1026*32]
      # print(x.shape)
      # Pass data through fc1
      # x = self.fc1(x)
      # print(x.shape)
      # x = F.relu(x)
      # print(x.shape)
      pred = self.fc2(x).reshape(dim0, -1) #[16,1026]

      return pred, x4

net = GraphNet()
criterion = nn.CrossEntropyLoss()
mse = nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [6]:
#import the libraries
import torch
from torch_geometric.data import Data

# #making the edge 
# #the tensor defining the source and target nodes of all edges, is not a list of index tuples
# edge_index = torch.tensor([[0, 1, 1, 2],
#                            [1, 0, 2, 1]], dtype=torch.long)
# #making nodes
# #Node feature matrix with shape [num_nodes, num_node_features]
# x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
# # edge_index = Graph connectivity in COO format with shape [2, num_edges]
# data = Data(x=x, edge_index=edge_index)

In [34]:

#optim = torch.optim.Adam(net.parameters(), lr=0.001)

In [8]:
!git clone https://github.com/fulifeng/Temporal_Relational_Stock_Ranking

Cloning into 'Temporal_Relational_Stock_Ranking'...
remote: Enumerating objects: 9134, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9134 (delta 0), reused 6 (delta 0), pack-reused 9126
Receiving objects: 100% (9134/9134), 330.32 MiB | 9.81 MiB/s, done.
Resolving deltas: 100% (46/46), done.
Checking out files: 100% (11378/11378), done.


In [9]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1fyNCZ62pEItTQYEBzLwsZ9ehX_-Ai3qT' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1fyNCZ62pEItTQYEBzLwsZ9ehX_-Ai3qT" -O sequential_embedding.tar.gz && rm -rf /tmp/cookies.txt

--2022-12-06 01:43:20--  https://docs.google.com/uc?export=download&confirm=t&id=1fyNCZ62pEItTQYEBzLwsZ9ehX_-Ai3qT
Resolving docs.google.com (docs.google.com)... 172.217.194.138, 172.217.194.139, 172.217.194.113, ...
Connecting to docs.google.com (docs.google.com)|172.217.194.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-bg-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/p8s6ltt1a5rf6gpaqipppf8k7a5c1u4h/1670290950000/07300192544028358737/*/1fyNCZ62pEItTQYEBzLwsZ9ehX_-Ai3qT?e=download&uuid=3902b25f-3872-4f87-8ae3-9e087984ff37 [following]
--2022-12-06 01:43:21--  https://doc-00-bg-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/p8s6ltt1a5rf6gpaqipppf8k7a5c1u4h/1670290950000/07300192544028358737/*/1fyNCZ62pEItTQYEBzLwsZ9ehX_-Ai3qT?e=download&uuid=3902b25f-3872-4f87-8ae3-9e087984ff37
Resolving doc-00-bg-docs.googleusercontent.com (doc-00-bg-docs.googleusercontent.com)... 142.251.12.1

In [10]:
!mv sequential_embedding.tar.gz Temporal_Relational_Stock_Ranking/data/
!tar -xvf ./Temporal_Relational_Stock_Ranking/data/sequential_embedding.tar.gz

pretrain/
pretrain/NASDAQ_rank_lstm_seq-16_unit-64_2.csv.npy
pretrain/NYSE_rank_lstm_seq-8_unit-32_0.csv.npy


In [11]:
!mv ./pretrain ./Temporal_Relational_Stock_Ranking/data/

In [12]:
!tar zxvf ./Temporal_Relational_Stock_Ranking/data/relation.tar.gz

relation/
relation/sector_industry/
relation/sector_industry/NASDAQ_industry_ticker.json
relation/sector_industry/NYSE_industry_ticker.json
relation/sector_industry/NASDAQ_industry_relation.npy
relation/sector_industry/NYSE_industry_relation.npy
relation/wikidata/
relation/wikidata/NASDAQ_wiki_relation.npy
relation/wikidata/NYSE_wiki_relation.npy
relation/wikidata/NYSE_connections.json
relation/wikidata/NASDAQ_connections.json
relation/wikidata/selected_wiki_connections.csv


In [13]:
!mv ./relation ./Temporal_Relational_Stock_Ranking/data/

In [14]:
import argparse
import copy
import numpy as np
import os
# import psutil
import random
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from time import time
try:
    from tensorflow.python.ops.nn_ops import leaky_relu
except ImportError:
    from tensorflow.python.framework import ops
    from tensorflow.python.ops import math_ops


    def leaky_relu(features, alpha=0.2, name=None):
        with ops.name_scope(name, "LeakyRelu", [features, alpha]):
            features = ops.convert_to_tensor(features, name="features")
            alpha = ops.convert_to_tensor(alpha, name="alpha")
            return math_ops.maximum(alpha * features, features)
# import sys  
# sys.path.insert(0, '/content/Temporal_Relational_Stock_Ranking')


In [15]:
!ln -s /content/Temporal_Relational_Stock_Ranking/training/load_data.py load_data.py
!ln -s /content/Temporal_Relational_Stock_Ranking/training/evaluator.py evaluator.py

In [16]:
seed = 123456789
np.random.seed(seed)
tf.random.set_seed(seed)

In [17]:

from load_data import load_EOD_data, load_relation_data
from evaluator import evaluate

In [19]:
desc = 'train a relational rank lstm model'
path = '/content/Temporal_Relational_Stock_Ranking/training/../data/2013-01-01'
market_name = 'NASDAQ'
length = 16
hidden_units = 64
steps = 1
lr = 0.001
alpha = 0.1
g_pu = 0
emb_file = 'NASDAQ_rank_lstm_seq-16_unit-64_2.csv.npy'
rel_name = 'wikidata'
inner_prod = 0
tickers = market_name + '_tickers_qualify_dr-0.98_min-5_smooth.csv'

parameters = {'seq': int(length), 'unit': int(hidden_units), 'lr': float(lr),
              'alpha': float(alpha)}
# print('arguments:', args)
print('parameters:', parameters)



parameters: {'seq': 16, 'unit': 64, 'lr': 0.001, 'alpha': 0.1}


In [20]:
em = np.load(os.path.join(path, '..', 'pretrain', emb_file))
print('embedding shape:', em.shape)
# print(em)

embedding shape: (1026, 1245, 64)


In [21]:
def load_EOD_data(data_path, market_name, tickers, steps=1):
    eod_data = []
    masks = []
    ground_truth = []
    base_price = []
    for index, ticker in enumerate(tickers):
        single_EOD = np.genfromtxt(
            os.path.join(data_path, market_name + '_' + ticker + '_1.csv'),
            dtype=np.float32, delimiter=',', skip_header=False
        )
        if market_name == 'NASDAQ':
            # remove the last day since lots of missing data
            single_EOD = single_EOD[:-1, :]
        if index == 0:
            print('single EOD data shape:', single_EOD.shape)
            eod_data = np.zeros([len(tickers), single_EOD.shape[0],
                                 single_EOD.shape[1] - 1], dtype=np.float32)
            masks = np.ones([len(tickers), single_EOD.shape[0]],
                            dtype=np.float32)
            ground_truth = np.zeros([len(tickers), single_EOD.shape[0]],
                                    dtype=np.float32)
            base_price = np.zeros([len(tickers), single_EOD.shape[0]],
                                  dtype=np.float32)
        for row in range(single_EOD.shape[0]):
            if abs(single_EOD[row][-1] + 1234) < 1e-8:
                masks[index][row] = 0.0
            elif row > steps - 1 and abs(single_EOD[row - steps][-1] + 1234) \
                    > 1e-8:
                ground_truth[index][row] = \
                    (single_EOD[row][-1] - single_EOD[row - steps][-1]) / \
                    single_EOD[row - steps][-1]
            for col in range(single_EOD.shape[1]):
                if abs(single_EOD[row][col] + 1234) < 1e-8:
                    single_EOD[row][col] = 1.1
        eod_data[index, :, :] = single_EOD[:, 1:]
        base_price[index, :] = single_EOD[:, -1]
    return eod_data, masks, ground_truth, base_price

In [22]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, path, emb_file, tickers, start, end):
        self.embedding = np.load(os.path.join(path, '..', 'pretrain', emb_file))[:, start:end, :]
        self.ticker = np.genfromtxt(os.path.join(path, '..', tickers), dtype=str, delimiter='\t', skip_header=False)
        _, self.mask, self.gt, self.price_data = load_EOD_data(path, 'NASDAQ', self.ticker)
        self.mask = self.mask[:, start:end]
        self.gt = self.gt[:, start:end]
        self.price_data = self.price_data[:, start:end]
        print('mask shape = ', self.mask.shape)
        print('gt shape = ', self.gt.shape)
        print('price_data shape = ', self.price_data.shape)

    def __len__(self):
        return len(self.embedding[0])

    def __getitem__(self, idx):
        emb = self.embedding[:, idx]
        label = self.gt[:, idx]
        price = self.price_data[:, idx]
        mask = self.mask[:, idx]
        return emb, label, price, mask

In [23]:
batch_size = 16

trainset = CustomDataset(path=path, emb_file=emb_file, tickers=tickers, start=0, end=756)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = CustomDataset(path=path, emb_file=emb_file, tickers=tickers, start=756, end=1008)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

single EOD data shape: (1245, 6)
mask shape =  (1026, 756)
gt shape =  (1026, 756)
price_data shape =  (1026, 756)
single EOD data shape: (1245, 6)
mask shape =  (1026, 252)
gt shape =  (1026, 252)
price_data shape =  (1026, 252)


In [24]:
import torch.optim as optim
from tqdm import tqdm

In [72]:

relation_encoding = np.load('/content/Temporal_Relational_Stock_Ranking/data/relation/sector_industry/NASDAQ_industry_relation.npy')

rel_shape = [relation_encoding.shape[0],relation_encoding.shape[1]]
# combine all the graphs to one graph
graph_flags = np.equal(np.zeros(rel_shape, dtype=int),
                          np.sum(relation_encoding, axis=2))
graph = np.where(graph_flags, np.zeros(rel_shape), np.ones(rel_shape))
# calculate the edge_index
edge_index = [[],[]]
for i in range(len(graph)):
  for j in range(len(graph[0])):
    if graph[i,j] == 1:
      edge_index[0].append(i)
      edge_index[1].append(j)



In [73]:
relation_encoding_wiki = np.load('/content/Temporal_Relational_Stock_Ranking/data/relation/wikidata/NASDAQ_wiki_relation.npy')
rel_shape_wiki = [relation_encoding_wiki.shape[0],relation_encoding_wiki.shape[1]]
# combine all the graphs to one graph
graph_flags_wiki = np.equal(np.zeros(rel_shape_wiki, dtype=int),
                          np.sum(relation_encoding_wiki, axis=2))
graph_wiki = np.where(graph_flags_wiki, np.zeros(rel_shape_wiki), np.ones(rel_shape_wiki))

for i in range(len(graph_wiki)):
  for j in range(len(graph_wiki[0])):
    if graph_wiki[i,j] == 1:
      edge_index[0].append(i)
      edge_index[1].append(j)

edge_index = np.array(edge_index)
print(edge_index.shape) #(2, 53612)

(2, 56575)


In [90]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"
edge_index = torch.tensor(edge_index).to(device)
for epoch in range(100):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in tqdm(enumerate(trainloader, 0)):
        # get the inputs; data is a list of [inputs, labels]
        # data = data.to(device)
        inputs, labels, price, mask = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        price = price.to(device)
        mask = mask.to(device)
        all_one = torch.ones((inputs.shape[0], 1)).to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        # print(inputs.get_device())
        # print(edge_index.get_device())
        # print(inputs.shape)
        outputs, output_feature = net(inputs, edge_index)
        # print(price.shape)
        return_ratio = torch.divide(torch.subtract(outputs, price), price)
        reg_loss = mse(labels,return_ratio)
        pre_pw_dif = torch.subtract(
            torch.matmul(return_ratio.T, all_one),
            torch.matmul(all_one.T, return_ratio)
        )
        gt_pw_dif = torch.subtract(
            torch.matmul(all_one.T, labels),
            torch.matmul(labels.T, all_one)
        )
        mask_pw = torch.matmul(mask.T, mask)
        a = F.relu( torch.multiply( torch.multiply(pre_pw_dif, gt_pw_dif), mask_pw ) )
        # print(a.numpy())
        rank_loss = np.mean(a.cpu().detach().numpy())
        loss = reg_loss + 0.1 * rank_loss
        # loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    
    print(f'[{epoch + 1}] loss: {running_loss}')


print('Finished Training')
torch.save(net.state_dict(), 'graph_model1_weight.pt')

<ipython-input-90-97246a776d6a>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.tensor(edge_index).to(device)
48it [00:11,  4.29it/s]

[1] loss: 43.72191862761974



48it [00:11,  4.23it/s]

[2] loss: 12.713349793106318



48it [00:11,  4.25it/s]

...

[98] loss: 2.1252084095031023



48it [00:21,  2.19it/s]

[99] loss: 2.1347595592960715



48it [00:21,  2.21it/s]


[100] loss: 2.12747361138463
Finished Training


In [91]:
def evaluate(prediction, ground_truth, mask, report=False):
    assert ground_truth.shape == prediction.shape, 'shape mis-match'
    performance = {}
    performance['mse'] = np.linalg.norm((prediction - ground_truth) * mask)**2\
        / np.sum(mask)
    mrr_top = 0.0
    all_miss_days_top = 0
    bt_long = 1.0
    bt_long5 = 1.0
    bt_long10 = 1.0

    for i in range(prediction.shape[1]):
        rank_gt = np.argsort(ground_truth[:, i])
        gt_top1 = set()
        gt_top5 = set()
        gt_top10 = set()
        for j in range(1, prediction.shape[0] + 1):
            cur_rank = rank_gt[-1 * j]
            if mask[cur_rank][i] < 0.5:
                continue
            if len(gt_top1) < 1:
                gt_top1.add(cur_rank)
            if len(gt_top5) < 5:
                gt_top5.add(cur_rank)
            if len(gt_top10) < 10:
                gt_top10.add(cur_rank)

        rank_pre = np.argsort(prediction[:, i])

        pre_top1 = set()
        pre_top5 = set()
        pre_top10 = set()
        for j in range(1, prediction.shape[0] + 1):
            cur_rank = rank_pre[-1 * j]
            if mask[cur_rank][i] < 0.5:
                continue
            if len(pre_top1) < 1:
                pre_top1.add(cur_rank)
            if len(pre_top5) < 5:
                pre_top5.add(cur_rank)
            if len(pre_top10) < 10:
                pre_top10.add(cur_rank)

        # calculate mrr of top1
        top1_pos_in_gt = 0
        for j in range(1, prediction.shape[0] + 1):
            cur_rank = rank_gt[-1 * j]
            if mask[cur_rank][i] < 0.5:
                continue
            else:
                top1_pos_in_gt += 1
                if cur_rank in pre_top1:
                    break
        if top1_pos_in_gt == 0:
            all_miss_days_top += 1
        else:
            mrr_top += 1.0 / top1_pos_in_gt

        # back testing on top 1
        real_ret_rat_top = ground_truth[list(pre_top1)[0]][i]
        bt_long += real_ret_rat_top

        # back testing on top 5
        real_ret_rat_top5 = 0
        for pre in pre_top5:
            real_ret_rat_top5 += ground_truth[pre][i]
        real_ret_rat_top5 /= 5
        bt_long5 += real_ret_rat_top5

        # back testing on top 10
        real_ret_rat_top10 = 0
        for pre in pre_top10:
            real_ret_rat_top10 += ground_truth[pre][i]
        real_ret_rat_top10 /= 10
        bt_long10 += real_ret_rat_top10


    performance['mrrt'] = mrr_top / (prediction.shape[1] - all_miss_days_top)
    performance['btl'] = bt_long
    # performance['btl5'] = bt_long5
    # performance['btl10'] = bt_long10
    return performance

In [92]:
cur_valid_pred = np.zeros([1026, 1008 - 756], dtype=float)
cur_valid_gt = np.zeros([1026, 1008 - 756], dtype=float)
cur_valid_mask = np.zeros([1026, 1008 - 756], dtype=float )
val_loss = 0.0
val_reg_loss = 0.0
val_rank_loss = 0.0
net.eval()
for i, data in tqdm(enumerate(testloader, 0)):
    # get the inputs; data is a list of [inputs, labels]
    # data = data.to(device)
    inputs, labels, price, mask = data
    inputs = inputs.to(device)
    labels = labels.to(device)
    price = price.to(device)
    mask = mask.to(device)
    all_one = torch.ones((inputs.shape[0], 1)).to(device)
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    
    outputs, output_feature = net(inputs, edge_index)
    return_ratio = torch.divide(torch.subtract(outputs, price), price)
    val_reg_loss = mse(labels,return_ratio)
    pre_pw_dif = torch.subtract(
        torch.matmul(return_ratio.T, all_one),
        torch.matmul(all_one.T, return_ratio)
    )
    gt_pw_dif = torch.subtract(
        torch.matmul(all_one.T, labels),
        torch.matmul(labels.T, all_one)
    )
    mask_pw = torch.matmul(mask.T, mask)
    a = F.relu( torch.multiply( torch.multiply(pre_pw_dif, gt_pw_dif), mask_pw ) )
    # print(a.numpy())
    val_rank_loss = np.mean(a.cpu().detach().numpy())
    val_loss = reg_loss + 0.1 * rank_loss

    # print statistics
print('val_loss: ', val_loss)
print('val_reg_loss: ', val_reg_loss)
print('val_rank_loss: ', val_rank_loss)

cur_valid_pred = copy.copy(return_ratio)
cur_valid_gt = copy.copy(labels)
cur_valid_mask = copy.copy(mask)

cur_valid_perf = evaluate(cur_valid_pred.cpu().detach().numpy(), cur_valid_gt.cpu().detach().numpy(), cur_valid_mask.cpu().detach().numpy())
print('Valid preformance:', cur_valid_perf)

16it [00:01,  8.32it/s]


val_loss:  tensor(0.0228, grad_fn=<AddBackward0>)
val_reg_loss:  tensor(0.0650, grad_fn=<MseLossBackward0>)
val_rank_loss:  1.6266454
Valid preformance: {'mse': 0.06486740544013098, 'mrrt': 0.21308437339139086, 'btl': -5.745768873835914}
